## Imports

In [1]:
import pandas as pd
import numpy as np
import datetime, random

from multiprocessing import Pool
import time

#Potentially useful 
#MRNs - ENC
#2307280 - 205472336

## Functions

In [9]:
def returnAKIpatients(df, 
                      aki_calc_type = 'rolling_window', keep_cols = True, 
                      cond1time = '48hours', cond2time = '168hours'):
    
    if aki_calc_type == 'both':
        df = df.groupby('enc', sort=False).apply(lambda d: addRollingWindowAKI(d,
                                                                              cond1time = cond1time,
                                                                              cond2time = cond2time))
        df = df.reset_index('enc', drop=True).reset_index()
        
        df = df.groupby('mrn', sort=False).apply(lambda d: addBaselineCreat(d))
        df = df.groupby('enc', sort=False).apply(lambda d: addBackCalcAKI(d))
        
    elif aki_calc_type == 'rolling_window':
        df = df.groupby('enc', sort=False).apply(lambda d: addRollingWindowAKI(d,
                                                                              cond1time = cond1time,
                                                                              cond2time = cond2time))
        
    elif aki_calc_type == 'back_calculate':
        df = df.groupby('mrn', sort=False).apply(lambda d: addBaselineCreat(d))
        df = df.groupby('enc', sort=False).apply(lambda d: addBackCalcAKI(d)) 
    
    return df

def addBaselineCreat(df):
    '''
    Adds the baseline creatinine to a dataframe. The baseline creatinine is defined as the median of the outpatient 
     creatinine values from 365 to 7 days prior to admission.
    
    Input: dataframe (typically of a single patient)
    Output: dataframe with baseline creatinine column added in
    '''
    t_m1y = (df.admission - datetime.timedelta(days=365)).values
    t_m7d = (df.admission - datetime.timedelta(days=7)).values
    
    df['baseline_creat'] = [df[~df.inpatient].set_index('time').sort_index().loc[t_m1y[indx]:t_m7d[indx]].creat.median() for indx in range(df.shape[0])]
    
    return df

def addBackCalcAKI(df):
    '''
    Adds the back-calculated AKI conditions, the KDIGO standards on the outpatient values;
     i.e. a 50% increase from baseline creatinine in <7 days
    
    Input: dataframe (typically of a single encounter)
    Output: dataframe with back-calculated aki values added in
    '''
    backcalc_aki = np.empty(df.shape[0])
    backcalc_aki[:] = np.nan
    
    df = df.sort_values('time')
    df_lf = df.set_index('time').loc[df.admission.values[0]:(df.admission + datetime.timedelta(days=7)).values[0]]
    backcalc_aki[:df_lf.shape[0]] = df_lf.creat > 1.5*df_lf.baseline_creat
    
    df['backcalc_aki'] = backcalc_aki
    
    return df 

def addRollingWindowAKI(df,
                        cond1time = '48hours', cond2time = '168hours'):
    '''
    Adds the AKI conditions based on rolling window definition: 0.3 creat increase in < 48 hrs OR 50% increase in < 7 days
    
    Input: dataframe (typically of a single encounter)
    Output: dataframe with rolling-window aki values added in
    '''
    df = df.set_index('time').sort_index()
    df = df[~df.duplicated()]
    
    df['mincreat_48hr'] = df.creat.rolling(pd.Timedelta(cond1time), min_periods=1).min().values
    df['mincreat_7day'] = df.creat.rolling(pd.Timedelta(cond2time), min_periods=1).min().values

    df['deltacreat_48hr'] = np.round(df.creat - df.mincreat_48hr, decimals = 3)
    df['deltacreat_7day'] = np.round(df.creat - df.mincreat_7day, decimals = 3)

    df['rollingwindow_aki'] = (df.deltacreat_48hr >= 0.3) | (df.deltacreat_7day > 0.5*df.mincreat_7day)
    
    return df

## Reading in file; managing columns

In [3]:
covid_df = pd.read_csv(r'H:\Data\Standardized AKI definition\dataset\covid creatinines.csv')
covid_df['mrn'] = covid_df.pat_mrn_id.str.strip('MR').astype('int')
covid_df['enc'] = covid_df.enc_id
covid_df['time'] = pd.to_datetime(covid_df.time)
covid_df['sex'] = covid_df.sex.astype('bool')
covid_df['race'] = covid_df.race.astype('bool')
covid_df['inpatient'] = covid_df.inpatient.astype('bool')
covid_df['creat'] = covid_df['creatinine']
covid_df['admission'] = pd.to_datetime(covid_df.admission)
covid_df['discharge'] = pd.to_datetime(covid_df.discharge)
print('Shape:', covid_df.shape)
print(covid_df.dtypes)

Shape: (299718, 14)
pat_mrn_id                object
pat_enc_csn_id             int64
time              datetime64[ns]
creatinine               float64
age                      float64
sex                         bool
race                        bool
enc_id                     int64
admission         datetime64[ns]
discharge         datetime64[ns]
inpatient                   bool
mrn                        int32
enc                        int64
creat                    float64
dtype: object


In [4]:
df = covid_df[['mrn', 'enc', 'time', 'creat', 'age', 'sex', 'race', 'inpatient', 'admission', 'discharge']]

In [10]:
%%time
out = returnAKIpatients(df, aki_calc_type = 'both', cond1time='52hours', cond2time='172hours')

Wall time: 15min 48s


## Adding Baseline Creat & Back-calculated AKI values

In [ ]:
%%time
#out_rw = returnAKIpatients(df, aki_calc_type = 'rolling_window')

In [ ]:
%%time
out_bc = returnAKIpatients(df, aki_calc_type = 'back_calculate')

In [ ]:
%%time
out = returnAKIpatients(df, aki_calc_type = 'both')

In [13]:
out.to_csv(r'H:\Data\Standardized AKI definition\dataset\output.csv')

## Two criterion for rolling-window definition of AKI:

#### *$creat \uparrow$ of 0.3 in < 48 hrs* OR *$creat \uparrow$ of 50% in < 7 days*

In [ ]:
def eGFR(creat, age, black, female):
    '''
    Calculates the estimated glomerular filtration rate based on the serum creatinine levels, age, sex, and race (black or not black);
    Based on the formula in the paper A New Equation to Estimate Glomerular Filtration Rate (Levey et. Al, 2009) linked below
    
    https://pubmed.ncbi.nlm.nih.gov/19414839/
    
    '''
    #Term 2 - np.clip(creat/(0.9-0.2*female, a_min=1, a_max=None) is the same as taking min(1, creat/k)
    #Term 3 - np.clip(creat/(0.9-0.2*female, a_min=None, a_max=None) is the same as taking max(1, creat/k)
    #where k is the data-derived constant given in the paper: 0.7 for females and 0.9 for males
    
    return 141*(np.clip(creat/(0.9-0.2*female), a_min=1, a_max=None)**(-0.411+0.082*female))*(np.clip(creat/(0.9-0.2*female), a_min=None, a_max=1)**-1.209)*(0.993**age)*(1+female*0.018)*(1+black*0.159)

#Sample test data
creat = np.random.normal(loc=1, scale=0.2, size=10)
age = np.random.normal(loc=55, scale=10, size=10)
black = np.random.rand(10) > 0.5
female = np.random.rand(10) > 0.5

eGFR(creat, age, black, female) 
#values seem pretty reasonable (80-120)

In [12]:
tmp = out.loc[out.enc == 205472336]
backcalc_aki = np.empty(tmp.shape[0])
backcalc_aki[:] = np.nan
tmp = tmp.sort_values('time')
tmp2 = tmp.set_index('time').sort_index().loc[tmp.admission.values[0]:(tmp.admission + datetime.timedelta(days=7)).values[0]]
backcalc_aki[:tmp2.shape[0]] = tmp2.creat > 1.5*tmp2.baseline_creat
tmp['backcalc_aki'] = backcalc_aki
tmp

time      mrn        enc  creat        age  \
enc                                                                         
205472336 63952 2019-09-23 20:19:00  2307280  205472336   0.51  37.605479   
          63953 2019-09-24 07:15:00  2307280  205472336   0.46  37.605479   
          63954 2019-09-25 08:20:00  2307280  205472336   0.50  37.605479   
          63955 2019-11-05 03:39:00  2307280  205472336   0.60  37.605479   
          63956 2019-11-05 07:37:00  2307280  205472336   0.90  37.605479   
          63957 2019-11-05 17:55:00  2307280  205472336   1.00  37.605479   
          63958 2019-11-06 08:04:00  2307280  205472336   1.20  37.605479   
          63959 2019-11-09 10:15:00  2307280  205472336   0.80  37.605479   
          63960 2019-11-19 07:44:00  2307280  205472336   0.91  37.605479   
          63961 2019-11-21 08:35:00  2307280  205472336   1.16  37.605479   
          63962 2019-11-22 08:46:00  2307280  205472336   1.45  37.605479   
          63963 2019-11-23 08:12:00  2307280  205472336   1.33  37.605479   
          63964 2019-11-24 08:13:00  2307280  205472336   1.50  37.605479   
          63965 2019-11-24 15:55:00  2307280  205472336   1.48  37.605479   
          63966 2019-11-25 08:17:00  2307280  205472336   1.54  37.605479   
          63967 2019-11-26 07:41:00  2307280  205472336   1.68  37.605479   
          63968 2019-11-27 17:58:00  2307280  205472336   1.40  37.605479   
          63969 2019-11-28 08:32:00  2307280  205472336   1.52  37.605479   
          63970 2019-11-29 07:44:00  2307280  205472336   2.12  37.605479   
          63971 2019-11-30 08:08:00  2307280  205472336   1.32  37.605479   
          63972 2019-12-02 08:11:00  2307280  205472336   1.90  37.605479   
          63973 2019-12-03 08:27:00  2307280  205472336   2.04  37.605479   
          63974 2019-12-04 08:24:00  2307280  205472336   2.52  37.605479   
          63975 2019-12-05 08:41:00  2307280  205472336   3.28  37.605479   
          63976 2019-12-08 08:38:00  2307280  205472336   3.88  37.605479   
          63977 2019-12-09 04:45:00  2307280  205472336   4.00  37.605479   
          63978 2019-12-09 08:49:00  2307280  205472336   4.72  37.605479   
          63979 2019-12-10 06:21:00  2307280  205472336   3.32  37.605479   
          63980 2019-12-11 09:11:00  2307280  205472336   4.52  37.605479   
          63981 2019-12-13 05:55:00  2307280  205472336   2.64  37.605479   
...                             ...      ...        ...    ...        ...   
          64045 2020-02-21 05:34:00  2307280  205472336   3.57  37.605479   
          64046 2020-02-22 07:07:00  2307280  205472336   4.75  37.605479   
          64047 2020-02-23 02:14:00  2307280  205472336   3.12  37.605479   
          64048 2020-02-23 06:29:00  2307280  205472336   3.34  37.605479   
          64049 2020-02-25 06:36:00  2307280  205472336   3.30  37.605479   
          64050 2020-02-26 08:00:00  2307280  205472336   4.34  37.605479   
          64051 2020-02-27 06:37:00  2307280  205472336   5.34  37.605479   
          64052 2020-02-27 22:00:00  2307280  205472336   5.91  37.605479   
          64053 2020-02-28 08:22:00  2307280  205472336   6.38  37.605479   
          64054 2020-03-03 07:25:00  2307280  205472336   4.93  37.605479   
          64055 2020-03-05 08:03:00  2307280  205472336   4.44  37.605479   
          64056 2020-03-06 08:19:00  2307280  205472336   3.29  37.605479   
          64057 2020-03-07 07:30:00  2307280  205472336   4.14  37.605479   
          64058 2020-03-08 07:58:00  2307280  205472336   3.02  37.605479   
          64059 2020-03-10 07:38:00  2307280  205472336   5.02  37.605479   
          64060 2020-03-13 07:35:00  2307280  205472336   3.59  37.605479   
          64061 2020-03-14 07:40:00  2307280  205472336   4.54  37.605479   
          64062 2020-03-15 08:04:00  2307280  205472336   3.24  37.605479   
          64063 2020-03-18 17:59:00  2307280  205472336   3.90  37.605479   
          64064 2020-03-19 